# Fan-Out Fan-In Edges with Multi-Agent Parallelization

This notebook demonstrates advanced **fan-out and fan-in patterns** using **AgentExecutor** to coordinate multiple AI agents working in parallel on the same task from different domain perspectives.

## Key Concepts

### Multi-Agent Parallelization
- **Multiple AI agents** process the same prompt simultaneously
- Each agent specializes in a different domain (research, marketing, legal)
- **AgentExecutor** integrates Azure OpenAI for LLM-powered processing
- Results are consolidated into a unified report

### Fan-Out with Target Routing
- `AgentExecutorRequest` includes `target_id` for agent selection
- Dispatcher creates multiple requests, each targeting a specific agent
- All agents receive the same prompt but apply domain-specific expertise

### Fan-In with Response Aggregation
- Collects `list[AgentExecutorResponse]` from all agents
- Each response contains agent insights and metadata
- Aggregator synthesizes insights into a consolidated report

## Workflow Architecture

```
DispatchToExperts (creates agent requests)
    ├──> researcher (AgentExecutor - research perspective)
    ├──> marketer (AgentExecutor - marketing perspective)
    └──> legal (AgentExecutor - legal perspective)
             ↓
         AggregateInsights (consolidates agent responses)
```

## What This Example Shows

1. **Multi-Agent Coordination**: Parallel execution of specialized AI agents
2. **Domain Expertise**: Each agent applies specific knowledge to the task
3. **Response Aggregation**: Consolidating diverse perspectives into unified output
4. **Azure OpenAI Integration**: Using AgentExecutor with cloud AI services
5. **Event Tracing**: AgentRunEvent for observability and debugging

## Setup

Import required modules and configure Azure OpenAI credentials.

### Prerequisites:
- Azure OpenAI deployment with GPT model
- Environment variables set:
  - `AZURE_OPENAI_API_KEY` or `AZURE_OPENAI_ENTRA_TOKEN`
  - `AZURE_OPENAI_ENDPOINT`
  - `AZURE_OPENAI_DEPLOYMENT_NAME`

In [ ]:
from dotenv import load_dotenv
import os
from dataclasses import dataclass

from agent_framework.workflows import Executor, ExecutorContext, Workflow
from agent_framework.workflows.openai import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    AgentRunEvent,
)
from azure.ai.projects.models import AzureAIClientConfiguration, ConnectionType
from openai.types.chat import ChatCompletionMessageParam

load_dotenv('../../.env')

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

# Verify Azure OpenAI configuration
required_vars = ["AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(f"⚠️  Missing environment variables: {', '.join(missing_vars)}")
    print("Please set them before running this notebook.")
else:
    print("✅ Azure OpenAI configuration found")

## Define Message Types

We'll use dataclasses to represent the workflow messages:

- **`Prompt`**: The user's question or task
- **`ConsolidatedReport`**: Aggregated insights from all agents

In [ ]:
@dataclass
class Prompt:
    """User prompt to be analyzed by multiple agents."""
    text: str


@dataclass
class ConsolidatedReport:
    """Consolidated insights from all domain expert agents."""
    insights: str

## Dispatcher Executor

The **DispatchToExperts** executor creates agent requests for each domain expert.

### Key Features:
- Converts prompt into `AgentExecutorRequest` objects
- Each request targets a specific agent via `target_id`
- All requests contain the same user prompt
- Fan-out ensures parallel agent execution

### Agent Targeting:
- `target_id="researcher"` → Research agent
- `target_id="marketer"` → Marketing agent
- `target_id="legal"` → Legal agent

In [ ]:
class DispatchToExperts(Executor[Prompt, AgentExecutorRequest]):
    """Dispatches the prompt to multiple domain expert agents in parallel."""

    async def execute(self, ctx: ExecutorContext[Prompt]) -> AgentExecutorRequest:
        prompt = ctx.get_input_data().text
        print(f"\n📋 Dispatching prompt to domain experts: '{prompt}'\n")

        # Create request with the user prompt
        messages: list[ChatCompletionMessageParam] = [
            {"role": "user", "content": prompt}
        ]

        # Return request - target_id will be set by fan-out routing
        return AgentExecutorRequest(messages=messages)

## Domain Expert Agents

We'll create three **AgentExecutor** instances, each with domain-specific instructions.

### Agent Roles:

1. **Researcher Agent** (`id="researcher"`)
   - Analyzes from academic and research perspective
   - Focuses on data, evidence, and scholarly insights

2. **Marketer Agent** (`id="marketer"`)
   - Evaluates from marketing and business perspective
   - Focuses on audience, positioning, and market impact

3. **Legal Agent** (`id="legal"`)
   - Reviews from legal and compliance perspective
   - Identifies risks, regulations, and legal considerations

### AgentExecutor Configuration:
- Uses Azure OpenAI for LLM inference
- System instructions define agent expertise
- Unique `id` enables targeted routing
- Handles `AgentExecutorRequest` → `AgentExecutorResponse` transformation

In [ ]:
# Configure Azure OpenAI client
client_config = AzureAIClientConfiguration(
    endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    type=ConnectionType.AZURE_OPEN_AI,
)

model = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]

# Create domain expert agents
researcher = AgentExecutor(
    id="researcher",
    client_config=client_config,
    model=model,
    instructions="You are a research expert. Analyze the prompt from an academic and research perspective. Provide data-driven insights.",
)

marketer = AgentExecutor(
    id="marketer",
    client_config=client_config,
    model=model,
    instructions="You are a marketing expert. Analyze the prompt from a marketing and business perspective. Focus on audience and market impact.",
)

legal = AgentExecutor(
    id="legal",
    client_config=client_config,
    model=model,
    instructions="You are a legal expert. Analyze the prompt from a legal and compliance perspective. Identify risks and regulations.",
)

print("✅ Domain expert agents created:")
print("   - Researcher (academic perspective)")
print("   - Marketer (business perspective)")
print("   - Legal (compliance perspective)")

## Aggregator Executor

The **AggregateInsights** executor consolidates responses from all domain experts.

### Key Features:
- Receives `list[AgentExecutorResponse]` from parallel agents
- Extracts text insights from each response
- Identifies agent source from response metadata
- Creates unified report with all perspectives

### Response Processing:
- Iterates through agent responses in order
- Extracts agent name from `target_id`
- Formats insights with clear attribution
- Combines into readable consolidated report

In [ ]:
class AggregateInsights(Executor[list[AgentExecutorResponse], ConsolidatedReport]):
    """Aggregates insights from all domain expert agents."""

    async def execute(
        self, ctx: ExecutorContext[list[AgentExecutorResponse]]
    ) -> ConsolidatedReport:
        responses = ctx.get_input_data()

        print("\n" + "="*60)
        print("AGGREGATING INSIGHTS FROM DOMAIN EXPERTS")
        print("="*60 + "\n")

        insights_text = []

        for response in responses:
            agent_name = response.request.target_id or "Unknown"
            insight = response.messages[-1].get("content", "") if response.messages else "No response"

            print(f"🔍 {agent_name.upper()} PERSPECTIVE:")
            print(f"{insight}\n")
            print("-" * 60 + "\n")

            insights_text.append(f"**{agent_name.capitalize()} Perspective:**\n{insight}")

        # Create consolidated report
        consolidated = "\n\n".join(insights_text)

        print("\n✅ All insights consolidated\n")
        return ConsolidatedReport(insights=consolidated)

## Build the Workflow

Construct the multi-agent workflow with fan-out and fan-in edges.

### Workflow Construction Steps:

1. **Create executor instances** (dispatcher, agents, aggregator)
2. **Add fan-out edges** from DispatchToExperts to all three agents
   - Automatically routes `AgentExecutorRequest` to matching `target_id`
3. **Add fan-in edges** from all agents to AggregateInsights
   - Collects `list[AgentExecutorResponse]` in order
4. **Set entry point** to DispatchToExperts

### Graph Visualization:
```
      DispatchToExperts
      /       |       \
 researcher marketer legal
      \       |       /
      AggregateInsights
```

In [ ]:
# Create executor instances
dispatcher = DispatchToExperts()
aggregator = AggregateInsights()

# Build the workflow
workflow = Workflow()

# Fan out from dispatcher to all domain expert agents
workflow.add_fan_out_edges(dispatcher, [researcher, marketer, legal])

# Fan in from all agents to aggregator
workflow.add_fan_in_edges([researcher, marketer, legal], aggregator)

# Set the entry point
workflow.set_entry_point(dispatcher)

print("✅ Multi-agent workflow constructed successfully!")
print("\nGraph structure:")
print("  DispatchToExperts → [researcher, marketer, legal] → AggregateInsights")

## Run the Workflow

Execute the workflow with a sample prompt and observe multi-agent collaboration.

### Expected Behavior:
1. Dispatcher sends prompt to all three agents
2. Agents process in parallel using Azure OpenAI
3. Each agent applies domain-specific expertise
4. Aggregator consolidates all perspectives
5. Final report includes research, marketing, and legal insights

### Note on Execution:
- Agents may complete in any order (parallel execution)
- Network latency affects individual agent completion time
- Framework ensures all agents complete before aggregation
- Event tracing via `AgentRunEvent` enables debugging

In [ ]:
# Define the prompt
user_prompt = "What are the implications of implementing AI-powered customer service chatbots?"

# Run the workflow
result = await workflow.run(Prompt(text=user_prompt))

print("\n" + "="*60)
print("FINAL CONSOLIDATED REPORT")
print("="*60 + "\n")
print(result.insights)

## Event Tracing with AgentRunEvent

The Agent Framework emits **AgentRunEvent** objects during execution for observability.

### Event Types:
- **`agent.run.started`**: Agent begins processing
- **`agent.run.completed`**: Agent finishes successfully
- **`agent.run.failed`**: Agent encounters error

### Event Data:
- `agent_id`: Which agent emitted the event
- `timestamp`: When the event occurred
- `request`: The input AgentExecutorRequest
- `response`: The output AgentExecutorResponse (if completed)
- `error`: Exception details (if failed)

### Use Cases:
- Debugging agent execution issues
- Performance monitoring and optimization
- Audit trails for compliance
- Real-time progress tracking

In [ ]:
# Example: Accessing event data programmatically
# (Events are automatically emitted during workflow execution)

print("\n📊 Agent Execution Events:")
print("   - Events are emitted during workflow.run()")
print("   - Use event handlers to capture and process events")
print("   - Enable telemetry for distributed tracing")
print("   - Integrate with Azure Monitor or Application Insights")

## Key Takeaways

### Multi-Agent Parallelization
- ✅ **Domain Expertise**: Each agent specializes in specific knowledge area
- ✅ **Parallel Execution**: All agents process simultaneously for efficiency
- ✅ **Diverse Perspectives**: Multiple viewpoints enrich the analysis
- ✅ **Consolidated Output**: Unified report combines all insights

### Fan-Out with AgentExecutor
- ✅ **Target Routing**: `target_id` directs requests to specific agents
- ✅ **Shared Prompt**: Same input analyzed from different angles
- ✅ **Independent Processing**: Agents don't interfere with each other
- ✅ **Scalable Pattern**: Easy to add more domain experts

### Fan-In with Response Aggregation
- ✅ **Type-Safe Collection**: `list[AgentExecutorResponse]` ensures consistency
- ✅ **Ordered Results**: Responses collected in edge definition order
- ✅ **Metadata Preservation**: Agent identity and request context maintained
- ✅ **Flexible Aggregation**: Custom logic for combining insights

### Azure OpenAI Integration
- ✅ **Enterprise-Ready**: Cloud-based LLM inference
- ✅ **Configuration Management**: Environment variables for credentials
- ✅ **Model Selection**: Specify deployment names for different models
- ✅ **System Instructions**: Define agent behavior and expertise

### Observability and Debugging
- ✅ **Event Tracing**: `AgentRunEvent` for execution monitoring
- ✅ **Error Handling**: Automatic failure detection and reporting
- ✅ **Performance Tracking**: Measure agent execution times
- ✅ **Audit Trails**: Record all agent interactions

### When to Use This Pattern
- ✅ Need diverse expert perspectives on the same problem
- ✅ Complex analysis requiring multiple knowledge domains
- ✅ Want to parallelize AI agent processing for speed
- ✅ Building multi-agent systems with specialized roles
- ✅ Require consolidated reporting from distributed agents

### Best Practices
- 🎯 **Define Clear Roles**: Give each agent specific expertise
- 🎯 **Use System Instructions**: Guide agent behavior with prompts
- 🎯 **Handle Failures Gracefully**: Implement error recovery strategies
- 🎯 **Monitor Performance**: Track agent execution times and costs
- 🎯 **Validate Credentials**: Check environment variables before execution
- 🎯 **Aggregate Thoughtfully**: Preserve attribution and context in reports

### Next Steps
- Add more domain experts (financial, technical, ethical, etc.)
- Implement dynamic agent selection based on prompt analysis
- Add retry logic for failed agent executions
- Integrate with Azure Monitor for production telemetry
- Experiment with different LLM models for each agent
- Build a web interface for interactive multi-agent consultation